In [ ]:
#| default_exp core
#| export

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from tqdm import tqdm
import time

df = pd.read_csv("vwo_scholen.csv")
school_data = df.set_index("school")

N_VWO = 2407  # exact aantal vwo-leerlingen 2025

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
import json, asyncio, secrets

labels_all = school_data.index.tolist()
MAX_SCHOLEN = 12

# Server-side opslag voor simulatieresultaten (te groot voor session cookie)
_sim_cache = {}

def get_kansen(volgorde):
    mat = simuleer_alle_lotnummers(volgorde, n_sim=500)
    n_sim, n = mat.shape
    n_scholen = len(volgorde)
    kansen = []
    for L in range(n):
        col = mat[:, L]
        rij = [(col == j).mean() * 100 for j in range(n_scholen)]
        rij.append((col == -1).mean() * 100)  # "Niet geplaatst"
        kansen.append(rij)
    return kansen, volgorde + ["Niet geplaatst"]

hdrs = [
    Script(src="https://d3js.org/d3.v7.min.js"),
    Script(src="https://cdn.jsdelivr.net/npm/sortablejs@1.15.0/Sortable.min.js"),
    Style("""
        :root {
            --si-bg: #e8edf2; --si-border: #c0c8d4; --si-text: #111;
            --muted: #666; --muted2: #aaa; --dim: #444; --school-text: #222;
            --card-bg: #fafafa; --card-border: #ddd; --border-light: #eee;
            --overlay-bg: #fff; --spinner-bg: rgba(255,255,255,0.7);
            --ghost-bg: #b0c4d8; --warn-bg: #fff8e1; --warn-text: #333;
            --tip-bg: rgba(255,255,255,0.97); --th-bg: #f0f0f0;
        }
        @media (prefers-color-scheme: dark) { :root {
            --si-bg: #2a2f36; --si-border: #444; --si-text: #ddd;
            --muted: #999; --muted2: #777; --dim: #bbb; --school-text: #ddd;
            --card-bg: #1e2228; --card-border: #444; --border-light: #333;
            --overlay-bg: #1e2228; --spinner-bg: rgba(0,0,0,0.6);
            --ghost-bg: #3a4550; --warn-bg: #3a3520; --warn-text: #ddd;
            --tip-bg: rgba(30,34,40,0.97); --th-bg: #2a2f36;
        }}

        svg text { fill: currentColor; }

        /* Lijst items */
        .school-item { padding:5px 8px; background:var(--si-bg); border:1px solid var(--si-border);
                       border-radius:5px; cursor:grab; list-style:none;
                       display:flex; align-items:center; gap:6px;
                       font-size:13px; color:var(--si-text); }
        .school-item:active { cursor:grabbing; }
        .school-nr { color:var(--muted); min-width:20px; font-size:12px; }
        .del-btn { margin-left:auto; cursor:pointer; color:var(--muted2); background:none;
                   border:none; font-size:15px; line-height:1; padding:0 2px; flex-shrink:0; }
        .del-btn:hover { color:#c0392b; }
        .sortable-ghost { opacity:0.35; background:var(--ghost-bg); }


        /* Spinner — volledig zelfstandig, niet afhankelijk van htmx-request class */
        #spinner { display:none; position:fixed; top:0; left:0; width:100%; height:100%;
                   background:var(--spinner-bg); z-index:9999;
                   align-items:center; justify-content:center; flex-direction:column; gap:16px; }
        #spinner.active { display:flex; }
        #spinner .spin-ring { width:52px; height:52px; border:5px solid var(--card-border);
                              border-top-color:#3498db; border-radius:50%;
                              animation:spin 0.75s linear infinite; }
        #spinner .spin-msg { font-size:15px; color:var(--dim); font-weight:500; }
        @keyframes spin { to { transform:rotate(360deg); } }

        /* School-selectie overlay */
        .school-overlay { display:none; position:fixed; top:0; left:0; width:100%; height:100%;
                          background:rgba(0,0,0,0.5); z-index:9998;
                          align-items:center; justify-content:center; }
        .school-overlay.active { display:flex; }
        .school-overlay-inner { background:var(--overlay-bg); color:var(--si-text); border-radius:12px;
                                max-width:700px; width:90%;
                                height:80vh; display:flex; flex-direction:column;
                                box-shadow:0 8px 32px rgba(0,0,0,0.25); overflow:hidden; }
        .school-overlay-header { padding:16px 20px; border-bottom:1px solid var(--border-light);
                                 display:flex; align-items:center; gap:12px; }
        .school-overlay-body { padding:12px 20px; overflow-y:auto; flex:1; min-height:0; }
        .school-overlay-inner form { display:flex; flex-direction:column; flex:1; min-height:0; }
        .school-overlay-footer { padding:12px 20px; border-top:1px solid var(--border-light);
                                 display:flex; justify-content:flex-end; gap:8px; }
        .school-check-item { display:flex; align-items:center; gap:6px; padding:4px 0;
                             font-size:13px; cursor:pointer; }
        .school-overlay-body > div { display:flex; flex-direction:column; }
        .school-check-item input[type="checkbox"] { margin:0; -webkit-appearance:checkbox;
                             appearance:checkbox; width:16px; height:16px; flex-shrink:0; }

        /* Cluster-badges */
        .badge { display:inline-block; padding:1px 6px; border-radius:10px;
                 font-size:10px; font-weight:600; margin-left:4px; opacity:0.9; }
        /* Cluster-overzicht tabel */
        .cluster-grid { display:grid; grid-template-columns:repeat(auto-fill,minmax(180px,1fr));
                        gap:10px; margin:12px 0; }
        .cluster-card { border:1px solid var(--card-border); border-radius:8px; padding:8px 10px;
                        background:var(--card-bg); }
        .cluster-card h5 { margin:0 0 6px 0; font-size:13px; font-weight:700;
                           display:flex; align-items:center; gap:6px; }
        .cluster-school { font-size:11px; color:var(--dim); padding:2px 0;
                          display:flex; align-items:center; gap:4px; }
        .cluster-school .pct { color:var(--muted); font-size:10px; }
        /* Chart schaalt mee */
        #chart svg { width:100%; height:auto; display:block; }
    """),
    *Theme.blue.headers()
]
app, rt = fast_app(hdrs=hdrs, secret_key="schoolkeuze-2026")

# d3 chart — viewBox zorgt voor schalen, geen vaste breedte
chart_js = """
function drawChart(DATA, LABELS){
    d3.select("#chart svg").remove();
    d3.select("#chart .tip").remove();
    const COLORS=["#4e79a7","#f28e2b","#e15759","#76b7b2","#59a14f","#edc948",
                  "#b07aa1","#ff9da7","#9c755f","#bab0ac","#d37295","#a0cbe8","#c0392b"];
    const M={top:30,right:185,bottom:45,left:55},W=900,H=420,
          iw=W-M.left-M.right,ih=H-M.top-M.bottom;
    const STK=DATA.map(row=>{const c=[0];for(let j=0;j<row.length;j++)c.push(c[j]+row[j]);return c;});
    const svg=d3.select("#chart").append("svg")
        .attr("viewBox",`0 0 ${W} ${H}`)
        .attr("preserveAspectRatio","xMidYMid meet")
        .append("g").attr("transform",`translate(${M.left},${M.top})`);
    const xS=d3.scaleLinear().domain([1,DATA.length]).range([0,iw]);
    const yS=d3.scaleLinear().domain([0,100]).range([ih,0]);
    const idx=d3.range(DATA.length);
    for(let j=0;j<LABELS.length;j++){
        svg.append("path").datum(idx)
            .attr("d",d3.area().curve(d3.curveMonotoneX)
                .x(i=>xS(i+1)).y0(i=>yS(STK[i][j])).y1(i=>yS(STK[i][j+1])))
            .attr("fill",COLORS[j]).attr("opacity",0.88);
    }
    svg.append("g").attr("transform",`translate(0,${ih})`)
        .call(d3.axisBottom(xS).ticks(8).tickFormat(d3.format("d")));
    svg.append("g").call(d3.axisLeft(yS).ticks(5).tickFormat(d=>d+"%"));
    svg.append("text").attr("x",iw/2).attr("y",ih+38).attr("text-anchor","middle")
        .style("font-size","12px").text("Lotnummer");
    svg.append("text").attr("x",iw/2).attr("y",-10).attr("text-anchor","middle")
        .style("font-size","14px").style("font-weight","600")
        .text("Plaatsingskans per school \u2014 voor elk lotnummer");
    const leg=svg.append("g").attr("transform",`translate(${iw+12},5)`);
    LABELS.forEach((l,i)=>{
        const g=leg.append("g").attr("transform",`translate(0,${i*22})`);
        g.append("rect").attr("width",11).attr("height",11).attr("rx",2).attr("fill",COLORS[i]);
        g.append("text").attr("x",15).attr("y",10).style("font-size","9px").text(l);
    });
    const hl=svg.append("line").attr("y1",0).attr("y2",ih)
        .attr("stroke","#333").attr("stroke-width",1.5)
        .attr("stroke-dasharray","5,3").style("opacity",0);
    const tip=d3.select("#chart").append("div").attr("class","tip")
        .style("position","absolute").style("background","var(--tip-bg)")
        .style("border","1px solid var(--card-border)").style("border-radius","8px")
        .style("padding","10px 14px").style("font-size","12px")
        .style("pointer-events","none").style("opacity",0)
        .style("box-shadow","0 3px 12px rgba(0,0,0,0.15)")
        .style("line-height","1.8").style("min-width","210px").style("color","var(--si-text)");
    svg.append("rect").attr("width",iw).attr("height",ih)
        .attr("fill","none").attr("pointer-events","all")
        .on("mousemove",function(event){
            const [mx]=d3.pointer(event);
            const lot=Math.max(1,Math.min(DATA.length,Math.round(xS.invert(mx))));
            hl.attr("x1",xS(lot)).attr("x2",xS(lot)).style("opacity",1);
            const row=DATA[lot-1];
            let h='<div style="font-size:13px;font-weight:700;margin-bottom:4px">Lotnummer '+lot+'</div>';
            row.forEach((v,i)=>{if(v>0.4)h+='<span style="color:'+COLORS[i]+'">\u25a0</span> '+LABELS[i]+': <b>'+v.toFixed(1)+'%</b><br>';});
            tip.html(h).style("opacity",1);
            const cr=document.getElementById("chart").getBoundingClientRect();
            let tx=event.clientX-cr.left+18;
            if(tx+230>cr.width)tx=event.clientX-cr.left-248;
            tip.style("left",tx+"px").style("top",(event.clientY-cr.top-20)+"px");
        })
        .on("mouseleave",()=>{hl.style("opacity",0);tip.style("opacity",0);});
}
"""

sortable_js = """
function initSortable(){
    const el=document.getElementById('sortable-list');
    if(el && !el._sortable){
        el._sortable=Sortable.create(el,{
            animation:120, ghostClass:'sortable-ghost',
            onEnd:()=>{ el.querySelectorAll('.school-nr').forEach((s,i)=>s.textContent=(i+1)+'.'); }
        });
    }
}
initSortable();
if(!window._sortableListenerAdded){
    document.body.addEventListener('htmx:afterSettle', initSortable);
    window._sortableListenerAdded=true;
}
"""

modal_check_js = """
function openSchoolModal() {
    document.getElementById('school-overlay').classList.add('active');
    document.body.style.overflow='hidden';
    updateCheckCount();
}
function closeSchoolModal() {
    document.getElementById('school-overlay').classList.remove('active');
    document.body.style.overflow='';
}
function updateCheckCount() {
    const checks = document.querySelectorAll('.school-check');
    const n = [...checks].filter(c => c.checked).length;
    document.getElementById('check-count').textContent = n;
    const full = n >= 12;
    checks.forEach(c => { if (!c.checked) c.disabled = full; });
    const btn = document.getElementById('modal-bevestig');
    if (btn) btn.disabled = n === 0;
    const cnt = document.getElementById('check-count');
    cnt.style.color = n === 12 ? '#2ecc71' : n > 0 ? '#e67e22' : '#aaa';
}
document.addEventListener('change', function(e) {
    if (e.target.classList.contains('school-check')) updateCheckCount();
});
document.body.addEventListener('htmx:afterSettle', function() {
    if (!document.querySelector('.school-overlay.active')) document.body.style.overflow='';
});
"""

# Kleur per cluster (badge + cluster-card header) — uitbreidbaar voor nieuwe clusters
CLUSTER_KLEUREN = {
    "gymnasium":    "#4e79a7",
    "traditioneel": "#59a14f",
    "montessori":   "#f28e2b",
    "vernieuwend":  "#e15759",
    "rest":         "#76b7b2",
}
_EXTRA_KLEUREN = ["#e15759","#b07aa1","#edc948","#9c755f","#ff9da7","#d37295","#a0cbe8"]

# --- School clusters ---
# Leerlingen kiezen niet willekeurig voor posities 4-12: wie een gymnasium kiest
# zet waarschijnlijk andere gymnasiums ook hoog. We gebruiken clusters om dit
# realistischer na te bootsen: scholen van hetzelfde type krijgen 3x meer kans
# om op de lagere lijstposities te verschijnen.
school_clusters = {
    "gymnasium":    ["Barlaeus Gymnasium", "Cygnus Gymnasium", "Vossius Gymnasium",
                     "Ignatiusgymnasium", "Het 4e Gymnasium",
                     "Montessori Lyceum Amsterdam - Gymnasium",
                     "Spinoza Lyceum - Gymnasium"],
    "traditioneel": ["Fons Vitae Lyceum", "Hyperion Lyceum", "Het Amsterdams Lyceum",
                     "Spinoza Lyceum", "Calandlyceum", "Ir. Lely Lyceum",
                     "St. Nicolaaslyceum", "St. Nicolaaslyceum - tto",
                     "Hervormd Lyceum Zuid", "Hervormd Lyceum West",
                     "Pieter Nieuwland College", "Pieter Nieuwland College - Plus",
                     "Damstede Lyceum", "Gerrit van der Veen College",
                     "Comenius Lyceum Amsterdam", "Berlage Lyceum - tto",
                     "Cartesius Lyceum - Het Lyceum"],
    "montessori":   ["Montessori Lyceum Amsterdam", "Montessori Lyceum Pax",
                     "Montessori Lyceum Terra Nova", "Metis Montessori Lyceum - Technasium",
                     "Metis Montessori Lyceum - Coderclass of Kunst & Co"],
    "vernieuwend":  ["Alasca", "Cartesius Lyceum - De Plaats", "DENISE",
                     "Geert Groote College", "Kairos Tienercollege",
                     "Lumion", "Marcanti College", "Metropolis Lyceum", "OSB",
                     "Vinse School", "Xplore"],
    "rest":         ["Cornelius Haga Lyceum"],
}

CLUSTERS_GEEN_BOOST = {"rest"}

def cluster_kleur(naam):
    if naam not in CLUSTER_KLEUREN:
        alle = list(school_clusters.keys())
        idx = alle.index(naam) if naam in alle else 0
        CLUSTER_KLEUREN[naam] = _EXTRA_KLEUREN[idx % len(_EXTRA_KLEUREN)]
    return CLUSTER_KLEUREN[naam]

school_naar_clusters = {}


def _cluster_gewichten(school, cluster_namen):
    """Geeft genormaliseerde gewichtsvector over alle clusters voor een school."""
    if school in school_naar_clusters:
        gewichten = school_naar_clusters[school]
    else:
        enkel = next((c for c, ss in school_clusters.items() if school in ss), "overig")
        gewichten = {enkel: 1.0}
    vec = np.zeros(len(cluster_namen))
    for c, w in gewichten.items():
        if c in cluster_namen:
            vec[cluster_namen.index(c)] = w
    s = vec.sum()
    return vec / s if s > 0 else vec

def simuleer_alle_lotnummers(mijn_voorkeuren, n_sim=500, n=2407):
    """
    Simuleert het DA-STD algoritme (v3) voor ALLE lotnummers tegelijk.
    Returns: matrix (n_sim, n) - per sim, per lotnummer: index in mijn_voorkeuren (0-11),
             -1 = niet geplaatst
    """
    scholen = school_data.index.tolist()
    n_s = len(scholen)
    caps_base = school_data["gpl_2025"].values.astype(int)
    max_via_k2  = school_data["nvk_2"].values.astype(int)
    max_via_k3  = school_data["nvk_3"].values.astype(int)
    max_via_k4p = school_data["nvk_4plus"].values.astype(int)
    max_via_k1  = (school_data["gpl_2025"] - school_data["nvk_2"] -
                   school_data["nvk_3"] - school_data["nvk_4plus"]).values.astype(int)
    p1 = (school_data["vk_1"] / school_data["vk_1"].sum()).values
    p2 = (school_data["vk_2"] / school_data["vk_2"].sum()).values
    p3 = (school_data["vk_3"] / school_data["vk_3"].sum()).values
    cluster_namen = list(school_clusters.keys())
    school_cluster_matrix = np.array([_cluster_gewichten(s, cluster_namen) for s in scholen])
    p_rest_per_cluster = np.zeros((len(cluster_namen), n_s))
    for ci in range(len(cluster_namen)):
        if cluster_namen[ci] in CLUSTERS_GEEN_BOOST:
            gew = np.ones(n_s)
        else:
            gew = 1.0 + 2.0 * school_cluster_matrix[:, ci]
        p_rest_per_cluster[ci] = gew / gew.sum()
    school_top_cluster = np.array([int(np.argmax(school_cluster_matrix[si])) for si in range(n_s)])
    log_p2 = np.zeros((n_s, n_s))
    for k in range(n_s):
        m = p2.copy(); m[k] = 0; s = m.sum()
        if s > 0: m /= s
        log_p2[k] = np.log(m + 1e-10)
    log_p3 = np.zeros((n_s, n_s, n_s))
    for ki in range(n_s):
        for kj in range(n_s):
            m = p3.copy(); m[ki] = 0; m[kj] = 0; s = m.sum()
            if s > 0: m /= s
            log_p3[ki, kj] = np.log(m + 1e-10)
    mijn_idx = np.array([scholen.index(s) for s in mijn_voorkeuren])
    n_anderen = n - 1
    arange_n = np.arange(n_anderen)
    plaatsing = np.full((n_sim, n), -1, dtype=np.int8)
    for sim in tqdm(range(n_sim), desc="Simuleren"):
        k1 = np.random.choice(n_s, size=n_anderen, p=p1)
        k2 = np.argmax(log_p2[k1] + np.random.gumbel(size=(n_anderen, n_s)), axis=1)
        k3 = np.argmax(log_p3[k1, k2] + np.random.gumbel(size=(n_anderen, n_s)), axis=1)
        k1_cluster = school_top_cluster[k1]
        log_p_rest = np.log(p_rest_per_cluster[k1_cluster] + 1e-10)
        scores = log_p_rest + np.random.gumbel(size=(n_anderen, n_s))
        scores[arange_n, k1] = np.inf
        scores[arange_n, k2] = np.inf - 1
        scores[arange_n, k3] = np.inf - 2
        pref_matrix = np.argsort(-scores, axis=1)
        cap = caps_base.copy()
        cnt_k1 = np.zeros(n_s, dtype=int)
        cnt_k2 = np.zeros(n_s, dtype=int)
        cnt_k3 = np.zeros(n_s, dtype=int)
        cnt_k4 = np.zeros(n_s, dtype=int)
        for j in range(len(mijn_idx)):
            if cap[mijn_idx[j]] > 0:
                plaatsing[sim, 0] = j; break
        for i in range(n_anderen):
            for school_idx in pref_matrix[i]:
                if cap[school_idx] <= 0: continue
                if school_idx == k1[i]:
                    if cnt_k1[school_idx] >= max_via_k1[school_idx]: continue
                    cnt_k1[school_idx] += 1
                elif school_idx == k2[i]:
                    if cnt_k2[school_idx] >= max_via_k2[school_idx]: continue
                    cnt_k2[school_idx] += 1
                elif school_idx == k3[i]:
                    if cnt_k3[school_idx] >= max_via_k3[school_idx]: continue
                    cnt_k3[school_idx] += 1
                else:
                    if cnt_k4[school_idx] >= max_via_k4p[school_idx]: continue
                    cnt_k4[school_idx] += 1
                cap[school_idx] -= 1
                break
            L = i + 2
            if L <= n:
                for j in range(len(mijn_idx)):
                    if cap[mijn_idx[j]] > 0:
                        plaatsing[sim, L-1] = j; break
    return plaatsing

def cluster_badges(naam):
    """Kleine gekleurde badges met cluster(s) voor een school."""
    if naam in school_naar_clusters:
        gewichten = school_naar_clusters[naam]
    else:
        enkel = next((c for c, ss in school_clusters.items() if naam in ss), "rest")
        gewichten = {enkel: 1.0}
    badges = []
    for cluster, w in sorted(gewichten.items(), key=lambda x: -x[1]):
        kleur = cluster_kleur(cluster)
        label = cluster if w == 1.0 else f"{cluster} {int(w*100)}%"
        badges.append(Span(label, cls="badge",
                           style=f"background:{kleur}22;color:{kleur};border:1px solid {kleur}66"))
    return badges

def school_item(naam, i):
    return Li(
        Span(f"{i+1}.", cls="school-nr"),
        Span(naam),
        *cluster_badges(naam),
        Button("✕", cls="del-btn", hx_post="/verwijder", hx_vals=f'{{"school":"{naam}"}}',
               hx_target="#lijst-panel", hx_swap="outerHTML"),
        Input(type="hidden", name="school", value=naam),
        cls="school-item", **{"data-id": naam}
    )

def _hoofd_cluster(s):
    if s in school_naar_clusters:
        return max(school_naar_clusters[s], key=school_naar_clusters[s].get)
    return next((c for c, ss in school_clusters.items() if s in ss), "overig")

def _cluster_dots(school):
    """Kleine gekleurde dots voor de cluster(s) van een school."""
    if school in school_naar_clusters:
        gewichten = school_naar_clusters[school]
    else:
        c = next((cl for cl, ss in school_clusters.items() if school in ss), "rest")
        gewichten = {c: 1.0}
    return [Span("●", style=f"color:{cluster_kleur(c)};font-size:11px",
                 title=f"{c} {int(w*100)}%" if w < 1 else c)
            for c, w in sorted(gewichten.items(), key=lambda x: -x[1])]

def school_modal(volgorde):
    """Overlay met alle scholen als checkboxes in een platte lijst met cluster-dots."""
    # Legenda bovenin
    legenda = Div(
        *[Span(Span("●", style=f"color:{cluster_kleur(c)};font-size:12px;margin-right:2px"),
               Span(c.capitalize(), style="font-size:11px;color:var(--muted);margin-right:10px"))
          for c in school_clusters],
        style="display:flex;flex-wrap:wrap;gap:4px;margin-bottom:8px"
    )
    checks = [Label(
        Input(type="checkbox", name="school", value=s,
              checked=s in volgorde, cls="school-check"),
        *_cluster_dots(s), Span(s),
        cls="school-check-item"
    ) for s in labels_all]
    cluster_sections = [Div(legenda, *checks)]

    n = len(volgorde)
    teller_kleur = "#2ecc71" if n == MAX_SCHOLEN else "#e67e22" if n > 0 else "#aaa"
    return Div(
        Div(
            Div(
                H4("🎓 Kies je scholen", style="margin:0"),
                Span(
                    Span(str(n), id="check-count",
                         style=f"font-weight:700;font-size:15px;color:{teller_kleur}"),
                    f" / {MAX_SCHOLEN}",
                    style="font-size:13px;color:var(--muted)"
                ),
                cls="school-overlay-header"
            ),
            Form(
                Div(*cluster_sections, cls="school-overlay-body"),
                Div(
                    Button("Annuleren", type="button", cls="uk-button uk-button-default",
                           onclick="closeSchoolModal()"),
                    Button("Bevestigen", type="submit", id="modal-bevestig",
                           cls="uk-button uk-button-primary"),
                    cls="school-overlay-footer"
                ),
                hx_post="/selecteer", hx_target="#lijst-panel", hx_swap="outerHTML",
                hx_on__after_request="closeSchoolModal()",
            ),
            cls="school-overlay-inner"
        ),
        id="school-overlay", cls="school-overlay",
        onclick="if(event.target===this)closeSchoolModal()"
    )

def lijst_panel(volgorde):
    n = len(volgorde)
    return Div(
        Card(
            H4("📋 Jouw voorkeurslijst"),
            P("Sleep om volgorde te wijzigen, ✕ om te verwijderen",
              cls="uk-text-muted", style="font-size:12px;margin-bottom:8px"),
            Form(
                Ul(*[school_item(s, i) for i, s in enumerate(volgorde)],
                   id="sortable-list",
                   style="padding:0;display:flex;flex-direction:column;gap:4px;min-height:30px"),
                Button("🔄 Herbereken", cls="uk-button uk-button-primary",
                       style="width:100%;margin-top:10px",
                       disabled=n != MAX_SCHOLEN,
                       title="" if n == MAX_SCHOLEN else f"Vul eerst {MAX_SCHOLEN} scholen in ({n}/{MAX_SCHOLEN})",
                       onclick="document.getElementById('spinner').classList.add('active')"),
                hx_post="/herbereken", hx_target="#chart-wrap", hx_swap="outerHTML", id="school-form",
                hx_on__after_request="document.getElementById('spinner').classList.remove('active')"
            ),
            Button("📝 Scholen kiezen", type="button",
                   cls="uk-button uk-button-default",
                   style="width:100%;margin-top:8px",
                   onclick="openSchoolModal()"),
        ),
        school_modal(volgorde),
        Script(NotStr(sortable_js)),
        Script(NotStr(modal_check_js)),
        id="lijst-panel"
    )

def school_tabel(volgorde):
    df_mijn = school_data.loc[volgorde]
    df_show = df_mijn[["cap_2025","gpl_2025","vk_1","vk_2","vk_3","nvk_1","nvk_2","nvk_3","nvk_4plus"]].copy()
    df_show.columns = ["Capaciteit","Geplaatst","1e keuze","2e keuze","3e keuze",
                       "Gepl. via 1e","Gepl. via 2e","Gepl. via 3e","Gepl. via 4e+"]
    html = (df_show.style
        .set_properties(**{'padding':'8px','text-align':'center'})
        .background_gradient(subset=["1e keuze","2e keuze","3e keuze"], cmap="YlOrRd")
        .background_gradient(subset=["Capaciteit","Geplaatst"], cmap="Blues")
        .set_caption("Jouw 12 VWO-scholen — Loting & Matching 2025")
        .set_table_styles([
            {'selector':'caption','props':[('font-size','16px'),('font-weight','bold'),('padding','10px')]},
            {'selector':'th','props':[('padding','8px'),('background-color','var(--th-bg)'),('color','var(--si-text)')]},
            {'selector':'th.row_heading','props':[('text-align','left')]},
            {'selector':'td','props':[('color','var(--si-text)')]},
            {'selector':'table','props':[('width','100%'),('border-collapse','collapse')]},
        ])
    ).to_html()
    return Div(NotStr(html), style="overflow-x:auto;margin-top:20px")

def _cluster_kaart_display(cluster, scholen_items, kleur):
    """Alleen-lezen cluster-kaart. scholen_items = [(school, w)] of [(nr, school, w)]"""
    rijen = []
    seen = set()
    for item in scholen_items:
        nr, school, w = item if len(item) == 3 else (None, item[0], item[1])
        if school in seen: continue
        seen.add(school)
        nr_span = Span(f"{nr}.", style="color:var(--muted2);font-size:10px;min-width:18px") if nr else NotStr("")
        pct = Span(f" {int(w*100)}%", style="font-size:9px;color:var(--muted2)") if w < 1.0 else NotStr("")
        rijen.append(Div(nr_span,
            Span("●", style=f"color:{kleur};font-size:9px"),
            Span(school, style="font-size:12px;color:var(--school-text)"),
            pct,
            style="display:flex;align-items:center;gap:5px;padding:2px 0"))
    n = len(seen)
    return Div(
        H5(Span("●", style=f"color:{kleur};font-size:13px"),
           f" {cluster.capitalize()}",
           Span(f" ({n})", style="color:var(--muted2);font-size:11px;font-weight:400"),
           style="margin:0 0 5px 0;font-size:13px;display:flex;align-items:center;gap:3px"),
        *rijen,
        cls="cluster-card", style=f"border-left:3px solid {kleur}")

def alle_clusters_panel():
    """Alle scholen gegroepeerd per cluster — puur weergave."""
    per_cluster = {c: [] for c in school_clusters}
    for school in labels_all:
        if school in school_naar_clusters:
            for c, w in school_naar_clusters[school].items():
                if c in per_cluster: per_cluster[c].append((school, w))
        else:
            c = next((cl for cl, ss in school_clusters.items() if school in ss), "rest")
            if c in per_cluster: per_cluster[c].append((school, 1.0))

    kaarten = [_cluster_kaart_display(c, items, cluster_kleur(c))
               for c, items in per_cluster.items()]

    return Div(
        H4("🗂️ Alle clusters — alle scholen", style="margin:0 0 4px 0;font-size:15px"),
        P("Clusters bepalen welke scholen vaker samen op voorkeurslijsten worden gezet. "
          "Scholen met meerdere clusters staan in elk relevant cluster met hun gewicht.",
          style="font-size:12px;color:var(--muted);margin:0 0 10px 0"),
        Div(*kaarten, cls="cluster-grid"),
        id="alle-clusters")

def cluster_overzicht_lijst(volgorde):
    """Alleen-lezen grid: jouw scholen gegroepeerd per cluster.
    Multi-cluster scholen verschijnen in elk cluster waar ze bij horen, met gewichts-badge."""
    # Bouw per cluster een lijst van (voorkeursnr, school, gewicht)
    per_cluster = {}
    for i, school in enumerate(volgorde):
        if school in school_naar_clusters:
            gewichten = school_naar_clusters[school]
        else:
            c = next((cl for cl, ss in school_clusters.items() if school in ss), "rest")
            gewichten = {c: 1.0}
        for c, w in gewichten.items():
            per_cluster.setdefault(c, []).append((i + 1, school, w))

    # Toon alleen clusters die minstens één van jouw scholen bevatten
    kaarten = []
    for cluster in school_clusters:  # behoud volgorde van school_clusters
        items = per_cluster.get(cluster, [])
        if not items: continue
        kleur = cluster_kleur(cluster)
        rijen = []
        for nr, school, w in items:
            pct = Span(f" {int(w*100)}%",
                       style="font-size:9px;color:var(--muted2)") if w < 1.0 else NotStr("")
            rijen.append(Div(
                Span(f"{nr}.", style="color:var(--muted2);font-size:10px;min-width:18px"),
                Span("●", style=f"color:{kleur};font-size:9px"),
                Span(school, style="font-size:12px;color:var(--school-text)"),
                pct,
                style="display:flex;align-items:center;gap:5px;padding:2px 0"
            ))
        kaarten.append(Div(
            H5(
                Span("●", style=f"color:{kleur};font-size:13px"),
                f" {cluster.capitalize()}",
                Span(f" ({len(items)})", style="color:var(--muted2);font-size:11px;font-weight:400"),
                style="margin:0 0 5px 0;font-size:13px;display:flex;align-items:center;gap:3px"
            ),
            *rijen,
            cls="cluster-card",
            style=f"border-left:3px solid {kleur}"
        ))

    return Div(
        H4("🗂️ Jouw scholen per cluster", style="margin:20px 0 8px 0;font-size:14px"),
        P("Scholen in hetzelfde cluster worden vaker samen op voorkeurslijsten gezet. "
          "Scholen met meerdere clusters staan in elk relevant cluster.",
          style="font-size:12px;color:var(--muted);margin:0 0 10px 0"),
        Div(*kaarten, cls="cluster-grid"),
    )

def results_ui(kansen, labels, volgorde):
    return Div(
        Div(id="chart", style="position:relative"),
        Script(NotStr(f"drawChart({json.dumps(kansen)},{json.dumps(labels)});")),
        A("🤖 Vraag AI om uitleg", href="/ai-prompt", target="_blank",
          cls="uk-button uk-button-default",
          style="margin-top:12px;display:inline-block"),
        cluster_overzicht_lijst(volgorde),
        school_tabel(volgorde),
        id="chart-wrap"
    )

def leeg_chart():
    return Div(
        P("👈 Voeg 12 scholen toe en klik Herbereken",
          style="text-align:center;color:var(--muted);padding:80px 0;font-size:16px"),
        id="chart-wrap"
    )

def uitleg():
    waarschuwing = Div(
        Strong("⚠️ Let op: "),
        Span("Dit is een "), Strong("simulatie"),
        Span(". De uitkomsten zijn gebaseerd op het DA-STD algoritme (Deferred Acceptance — "
             "Single Tie-breaking) dat Amsterdam gebruikt voor de VWO-loting & matching. "
             "We werken met "), Strong("fictieve voorkeurslijsten "),
             Span("die statistisch overeenkomen met de werkelijke verdeling uit "),
             A("Tabel 17 van het Loting-en-Matching 2025 verslag (OSVO/Mapping Worlds)",
               href="https://www.osvo.nl/loting-en-matching", target="_blank"),
             Span(". De individuele combinaties die leerlingen werkelijk maken zijn niet bekend, "
                  "waardoor de precieze verdeling per ronde afwijkt — maar de "
                  "plaatsingskansen per school zijn wel betrouwbaar."),
        style="background:var(--warn-bg);border-left:4px solid #f39c12;padding:10px 14px;"
              "border-radius:4px;font-size:13px;color:var(--warn-text);margin-top:8px"
    )

    hoe_werkt_het = Details(
        Summary(Strong("📖 Hoe werkt de simulatie?"), style="cursor:pointer;font-size:14px;margin-top:12px"),
        Div(
            H4("Wat proberen we te berekenen?", style="margin-top:12px"),
            P("We willen weten: ", Strong("als jij lotnummer X krijgt, op welke school kom je dan terecht?"),
              " Omdat we jouw lotnummer van tevoren niet weten, berekenen we de kans voor elk mogelijk lotnummer (1 tot 2407)."),

            H4("Stap 1: Andere leerlingen nabootsen"),
            P("In werkelijkheid hebben alle 2407 leerlingen een eigen gerangschikte lijst van scholen ingediend. "
              "Die lijsten hebben we niet. We maken ze na op basis van wat we wél weten:"),
            Ul(
                Li(Strong("1e keuze"), ": we weten van elke school hoeveel leerlingen die als eerste keuze hadden. "
                   "Daarmee maken we een kansenverdeling."),
                Li(Strong("2e keuze"), ": idem, gebaseerd op de werkelijke tweede-keuze aantallen — maar altijd anders dan de eerste keuze."),
                Li(Strong("3e keuze"), ": zelfde aanpak."),
                Li(Strong("4e tot 12e keuze"), ": hier hebben we geen data meer. We gebruiken ", Strong("clusters"), " (zie hieronder)."),
            ),

            H4("Stap 1b: Clusters — scholen van hetzelfde type"),
            P("Leerlingen zetten niet willekeurig scholen op positie 4 tot 12. Wie Barlaeus Gymnasium als eerste "
              "keuze heeft, zet waarschijnlijk ook andere gymnasiums hoog op zijn lijst — niet ineens een "
              "Montessori school. We hebben de 41 scholen ingedeeld in vijf groepen op basis van onderwijstype "
              "en schoolcultuur:"),
            Ul(
                Li(Strong("Gymnasium"), " (7): klassieke vorming met Latijn/Grieks — Barlaeus, Cygnus, Vossius, "
                   "Ignatiusgymnasium, Het 4e Gymnasium, Spinoza Gymnasium, Montessori Gymnasium"),
                Li(Strong("Traditioneel lyceum"), " (17): gevestigde lycea met een klassiek profiel — Het Amsterdams "
                   "Lyceum, Fons Vitae, Hyperion, Spinoza, Calandlyceum, Ir. Lely, St. Nicolaaslyceum (+tto), "
                   "Hervormd Zuid/West, Pieter Nieuwland (+Plus), Damstede, Gerrit van der Veen, Comenius, "
                   "Berlage-tto, Cartesius Het Lyceum"),
                Li(Strong("Montessori"), " (5): scholen met de Montessori-pedagogiek — Montessori Amsterdam, "
                   "Pax, Terra Nova, Metis Technasium, Metis Kunst & Co"),
                Li(Strong("Vernieuwend"), " (11): nieuwere scholen met een eigen onderwijsconcept — Alasca, "
                   "Cartesius De Plaats, DENISE, Geert Groote, Kairos, Lumion, Marcanti, Metropolis, OSB, "
                   "Vinse, Xplore"),
                Li(Strong("Rest"), " (1): Cornelius Haga — past bij geen enkel ander cluster"),
            ),
            P(Strong("Waarom geen levensbeschouwelijk cluster? "),
              "Hoewel er zowel christelijke als islamitische scholen zijn, kiezen ouders die specifiek "
              "islamitisch onderwijs willen niet voor een christelijke school en vice versa. Daarom zijn "
              "deze scholen ingedeeld bij het cluster dat past bij hun type onderwijs."),
            P("Scholen uit hetzelfde cluster krijgen een ", Strong("drie keer hogere kans"),
              " om op posities 4–12 te verschijnen. ", Strong("Uitzondering: "),
              "het rest-cluster krijgt géén boost (factor 1×), omdat er geen verwantschap is met andere scholen."),

            H4("Stap 2: De loting uitvoeren"),
            P("Nu simuleren we het echte algoritme:"),
            Ol(
                Li("We geven alle 2407 leerlingen een willekeurig lotnummer."),
                Li("We beginnen bij lotnummer 1 en gaan omhoog."),
                Li("Elke leerling krijgt de hoogste school op zijn lijst waar nog een plek vrij is."),
                Li("Is een school vol? Dan schuift die leerling door naar zijn volgende keuze."),
                Li("We houden bij: als ", Strong("jij"), " nu aan de beurt was, op welke school zou je terechtkomen?"),
            ),

            H4("Stap 3: Extra rem op de cascade"),
            P("Sommige scholen hebben in werkelijkheid ", Strong("alle plekken gevuld via eerste keuze"),
              ". We gebruiken de werkelijke data om een harde grens in te stellen: "
              "per school laten we nooit meer plekken via een bepaalde voorkeurspositie vullen dan in werkelijkheid is gebeurd."),

            H4("Stap 4: Herhalen voor zekerheid"),
            P("Eén simulatie geeft al een antwoord, maar door toeval kan dat soms afwijken. "
              "Daarom herhalen we het hele proces ", Strong("500 keer"),
              ". Uiteindelijk middelen we de uitkomsten."),

            H4("Beperkingen"),
            Ul(
                Li("We weten niet wie welke combinatie kiest — alleen de totalen per school."),
                Li("De cluster-indeling en de factor 3× zijn schattingen, geen gemeten waarden."),
                Li("Keuzes 4 tot 12 zijn onzeker — de cluster-aanpak is beter dan willekeurig, maar nog steeds een vereenvoudiging."),
                Li("We gebruiken cijfers uit 2025 voor 2026 — capaciteit en populariteit kunnen veranderen."),
                Li("Bijzondere regels (voorrang, Kopklas) tellen niet mee (max. 2% van de plekken)."),
            ),
            P(Strong("Conclusie: "), "de simulatie geeft een goede indicatie van je kansen, maar is geen exacte voorspelling. "
              "Gebruik het als hulpmiddel bij het nadenken over je lijstvolgorde, niet als garantie."),
            style="font-size:13px;color:var(--warn-text);line-height:1.7;padding:8px 0"
        ),
    )

    return Card(
        H3("🎓 Schoolkeuze Simulator — Amsterdam VWO 2026"),
        P("Met deze simulator kun je zien wat jouw kansen zijn op plaatsing bij VWO-scholen in Amsterdam, "
          "afhankelijk van je lotnummer. Stel je voorkeurslijst samen, klik op Herbereken, en de grafiek "
          "laat zien bij welke school je terechtkomt voor elk mogelijk lotnummer (1–2407)."),
        waarschuwing,
        hoe_werkt_het,
        style="margin-bottom:16px"
    )

@rt("/")
def get(session):
    volgorde = session.get("volgorde", [])
    return Titled("Schoolkeuze Simulator",
        Script(NotStr(chart_js)),
        Div(Div(cls="spin-ring"), Div("Simulatie loopt...", cls="spin-msg"), id="spinner"),
        uitleg(),
        lijst_panel(volgorde),
        Card(
            P("Beweeg over de grafiek voor details per lotnummer", cls="uk-text-muted"),
            leeg_chart(),
        ),
        Script(NotStr('htmx.trigger(document.getElementById("school-form"),"submit");'))
            if len(volgorde) == MAX_SCHOLEN else NotStr(""),
        Card(alle_clusters_panel(), style="margin-top:24px"),
    )

@rt("/selecteer")
async def post(req, session):
    form = await req.form()
    nieuwe = form.getlist("school")
    oude = list(session.get("volgorde", []))
    # Behoud volgorde van bestaande, voeg nieuwe toe aan eind
    volgorde = [s for s in oude if s in nieuwe] + [s for s in nieuwe if s not in oude]
    volgorde = volgorde[:MAX_SCHOLEN]
    session["volgorde"] = volgorde
    return lijst_panel(volgorde)

@rt("/verwijder")
async def post(req, session):
    form = await req.form()
    school = form.get("school")
    volgorde = [s for s in session.get("volgorde", []) if s != school]
    session["volgorde"] = volgorde
    return lijst_panel(volgorde)

@rt("/herbereken")
async def post(req, session):
    form = await req.form()
    volgorde = form.getlist("school")
    session["volgorde"] = volgorde
    if len(volgorde) != MAX_SCHOLEN: return leeg_chart()
    k, lbl = await asyncio.to_thread(get_kansen, volgorde)
    sid = session.setdefault("sid", secrets.token_hex(8))
    _sim_cache[sid] = {"kansen": k, "labels": lbl}
    return results_ui(k, lbl, volgorde)

import io, base64

GITHUB_URL = "https://github.com/daburer/schoolkeuze2026"
SAMPLE_LOTS = [100, 500, 1000, 1500, 2000]

def maak_ai_prompt(volgorde, kansen, labels):
    """Bouw een kopieerbare prompt met alle simulatiegegevens."""
    lines = [
        "# Verklaar deze simulatie-uitkomsten",
        "",
        "Hieronder vind je de resultaten van een simulatie van de Amsterdamse VWO-loting & matching (DA-STD algoritme).",
        f"De broncode staat op: {GITHUB_URL}",
        "",
        "## Voorkeurslijst",
        *[f"{i+1}. {s}" for i, s in enumerate(volgorde)],
        "",
        "## Clusterindeling",
        "Scholen in hetzelfde cluster krijgen 3× meer kans op posities 4–12 van gesimuleerde voorkeurslijsten.",
        "Het rest-cluster krijgt géén boost (factor 1×).",
        "",
    ]
    for cluster, scholen in school_clusters.items():
        boost = "1×" if cluster in CLUSTERS_GEEN_BOOST else "3×"
        lines.append(f"- **{cluster.capitalize()}** ({boost}): {', '.join(scholen)}")
    lines += ["", "## Plaatsingskansen per lotnummer", ""]

    # Tabelheader
    header = "| School | " + " | ".join(f"Lot {l}" for l in SAMPLE_LOTS) + " |"
    sep = "|---|" + "|".join("---:" for _ in SAMPLE_LOTS) + "|"
    lines += [header, sep]
    for j, label in enumerate(labels):
        vals = " | ".join(f"{kansen[l-1][j]:.1f}%" for l in SAMPLE_LOTS)
        lines.append(f"| {label} | {vals} |")

    lines += [
        "",
        "## Vraag",
        "Verklaar deze uitkomsten. Waarom zijn de kansen zo verdeeld? "
        "Welke scholen zijn risicovol en welke zijn veilig? "
        "Geef advies over de lijstvolgorde.",
    ]
    return "\n".join(lines)

def maak_chart_png(kansen, labels):
    """Genereer een stacked area chart als base64-encoded PNG."""
    n_lots = len(kansen)
    n_labels = len(labels)
    x = np.arange(1, n_lots + 1)
    data = np.array(kansen)

    fig, ax = plt.subplots(figsize=(10, 5))
    colors = ["#4e79a7","#f28e2b","#e15759","#76b7b2","#59a14f","#edc948",
              "#b07aa1","#ff9da7","#9c755f","#bab0ac","#d37295","#a0cbe8","#c0392b"]
    ax.stackplot(x, data.T, labels=labels, colors=colors[:n_labels], alpha=0.88)
    ax.set_xlim(1, n_lots)
    ax.set_ylim(0, 100)
    ax.set_xlabel("Lotnummer")
    ax.set_ylabel("%")
    ax.set_title("Plaatsingskans per school — voor elk lotnummer")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    ax.legend(loc="upper left", bbox_to_anchor=(1.01, 1), fontsize=8)
    fig.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode()

@rt("/ai-prompt")
def get(session):
    volgorde = session.get("volgorde", [])
    sid = session.get("sid")
    cached = _sim_cache.get(sid) if sid else None
    if not cached or len(volgorde) != MAX_SCHOLEN:
        return Titled("AI Prompt", P("Geen simulatie gevonden. Ga terug en klik eerst op Herbereken.",
                                     style="padding:40px;text-align:center"))
    kansen, labels = cached["kansen"], cached["labels"]

    prompt_text = maak_ai_prompt(volgorde, kansen, labels)
    chart_b64 = maak_chart_png(kansen, labels)

    return Titled("🤖 AI Prompt — Simulatie uitleg",
        Script(NotStr(chart_js)),
        Style("textarea{font-family:monospace;font-size:12px;white-space:pre;}"
              ".copy-btn{cursor:pointer;transition:background 0.2s}"
              ".copy-btn.copied{background:#2ecc71!important;color:#fff}"),
        Card(
            P("Kopieer onderstaande tekst en plak het in een AI (ChatGPT, Claude, etc.) "
              "om uitleg te krijgen over je simulatie-uitkomsten. "
              "Je kunt ook de grafiek als afbeelding meesturen.",
              style="font-size:13px;color:var(--muted);margin-bottom:12px"),
            H4("📊 Grafiek"),
            Img(src=f"data:image/png;base64,{chart_b64}",
                style="width:100%;max-width:900px;border-radius:8px;margin-bottom:16px"),
            P("Rechtermuisklik → Afbeelding opslaan om mee te sturen naar een AI.",
              style="font-size:11px;color:var(--muted2);margin-bottom:16px"),
            H4("📋 Prompt"),
            Textarea(prompt_text, id="ai-prompt-text", readonly=True,
                     style="width:100%;height:400px;padding:12px;border-radius:8px;"
                           "border:1px solid var(--card-border);background:var(--card-bg);"
                           "color:var(--si-text);resize:vertical"),
            Button("📋 Kopieer prompt", cls="uk-button uk-button-primary copy-btn",
                   style="margin-top:10px",
                   onclick="navigator.clipboard.writeText(document.getElementById('ai-prompt-text').value)"
                           ".then(()=>{const b=this;b.textContent='✅ Gekopieerd!';b.classList.add('copied');"
                           "setTimeout(()=>{b.textContent='📋 Kopieer prompt';b.classList.remove('copied')},2000)})"),
            A("← Terug naar simulator", href="/", cls="uk-button uk-button-default",
              style="margin-top:10px;margin-left:8px"),
        ),
    )

serve()